In [1]:
from google.colab import files
import pathlib
import matplotlib.pyplot as plt
import sys
import torch
import numpy as np
import pandas as pd
import math
from torch.autograd import Variable
import torch.nn as nn
%matplotlib inline 

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
gd_folder = "/content/gdrive/My Drive/Colab Notebooks/HGCN/"
sys.path.append(gd_folder)
from modules import FilterLinear,GraphConvolutionalLSTM,TrainModel,TestModel,PrepareDataset

In [4]:
print(torch.cuda.is_available())

True


In [ ]:
pathlib.Path(gd_folder+"out").mkdir(parents=True, exist_ok=True) 
speed_matrix =  pd.read_csv(gd_folder+"data/my_data.csv")
K = 3
Clamp_A = False
s=len(speed_matrix.columns)
all = [torch.eye(s),torch.eye(s),torch.eye(s)]
fs_fold = "EnsRanking/dynamic/"
for horizon in [1,6,12]:
  MSEs = []
  sl = int(24*7*12/0.8979)
  ran = range(0,24*7*12,12)
  gclstm = GraphConvolutionalLSTM(K, torch.ones([s,s]), all, s, Clamp_A=Clamp_A, output_last = True)
  for i in ran:
        directory = gd_folder+'data/'+fs_fold+str(i+sl)+"/"
        FFR_5min = np.loadtxt(open(directory + "l1.csv", "rb"), delimiter=",", skiprows=0)
        FFR_10min = np.loadtxt(open(directory + "l2.csv", "rb"), delimiter=",", skiprows=0)
        FFR_15min = np.loadtxt(open(directory + "l3.csv", "rb"), delimiter=",", skiprows=0)
        FFR = [torch.Tensor(FFR_5min)+torch.eye(FFR_5min.shape[0],FFR_5min.shape[1]), torch.Tensor(FFR_10min), torch.Tensor(FFR_15min)]
        setattr(gclstm,'A_list',FFR)
        gclist=getattr(gclstm,'gc_list')
        g1=gclist[0]
        setattr(g1,'filter_square_matrix',Variable(FFR[0].cuda(), requires_grad=False))
        g2=gclist[1]
        setattr(g2,'filter_square_matrix',Variable(FFR[1].cuda(), requires_grad=False))
        g3=gclist[2]
        setattr(g3,'filter_square_matrix',Variable(FFR[2].cuda(), requires_grad=False))
        setattr(gclstm,'gc_list',nn.ModuleList([g1,g2,g3]))
        dat=speed_matrix.iloc[i:i+sl]
        max_speed = dat.max().max()
        train_dataloader, valid_dataloader, test_dataloader, max_speed = PrepareDataset(dat,BATCH_SIZE=40, pred_len = horizon,seq_len = 3)
        gclstm, gclstm_loss = TrainModel(gclstm, train_dataloader, valid_dataloader, num_epochs = 1000)
        gclstm_test = TestModel(gclstm, test_dataloader, max_speed )
        err = math.sqrt(gclstm_test[1][0])*max_speed
        MSEs.append(err)
        print(i," of ", max(ran),"; MSE=",err)
        #if i % 100 == 0:
        #  np.save('/content/gdrive/My Drive/projects/Graph_Convolutional_LSTM/out/CCF/dynamic/MSE'+str(horizon)+"-"+str(i), MSEs)
        #  print(np.mean(MSEs))
  pathlib.Path(gd_folder+'out/'+fs_fold).mkdir(parents=True, exist_ok=True) 
  np.save(gd_folder+'out/'+fs_fold+'MSEs'+str(horizon), MSEs)

fold = gd_folder+'out/'+fs_fold
print("h=1; MSE =",round(np.mean(np.load(fold+'MSEs1.npy')),2))
print("h=6; MSE =",round(np.mean(np.load(fold+'MSEs6.npy')),2))
print("h=12; MSE =",round(np.mean(np.load(fold+'MSEs12.npy')),2))

Early Stopped at Epoch: 757
0  of  2004 ; MSE= 6.952747214954147
Early Stopped at Epoch: 420
12  of  2004 ; MSE= 5.557072713820585
Early Stopped at Epoch: 342
24  of  2004 ; MSE= 4.8508257933084185


In [ ]:
fs_fold = "EnsVoting/dynamic/"
fold = gd_folder+'out/'+fs_fold
print("h=1; MSE =",round(np.mean(np.load(fold+'MSEs1.npy')),2))
print("h=6; MSE =",round(np.mean(np.load(fold+'MSEs6.npy')),2))
print("h=12; MSE =",round(np.mean(np.load(fold+'MSEs12.npy')),2))

h=1; MSE = 15.51
h=6; MSE = 17.02
h=12; MSE = 17.91


In [ ]:
speed_matrix =  pd.read_csv(gd_folder+"data/my_data.csv")
max_speed = speed_matrix.max().max()
s=len(speed_matrix.columns)
K = 3
Clamp_A = False
sl = int(24*7*12/0.8979)
dat=speed_matrix.iloc[0:sl]
train_dataloader, valid_dataloader, test_dataloader, max_speed = PrepareDataset(dat,BATCH_SIZE=40, pred_len = 1,seq_len = 3)
all = [torch.eye(s),torch.eye(s),torch.eye(s)]
gclstm = GraphConvolutionalLSTM(K, torch.ones([s,s]), all, s, Clamp_A=Clamp_A, output_last = True)
gclstm, gclstm_loss = TrainModel(gclstm, train_dataloader, valid_dataloader, num_epochs = 1000)
gclstm_test = TestModel(gclstm, test_dataloader, max_speed )

Early Stopped at Epoch: 849
